In [ ]:
workdir = "/content/drive/MyDrive/QA_pt"
%mkdir {workdir}
!export HF_DATASETS_CACHE="{workdir}/cache_huggingface"

cache_dir=f"{workdir}/cache_huggingface"

%cd {workdir}

!apt update -y
!apt install -y enchant libenchant-dev  hunspell-pt-br
!pip install pyenchant datasets pycountry

/content/drive/MyDrive/QA_pt
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ge

In [ ]:
from datasets import load_dataset
import enchant
import re

def is_portuguese(text: str) -> bool:
  words = re.split(r'\s', text)
  words = [w for w in words if w.isalpha()]
  pt_words = [w for w in words if spell.check(w)]

  return len(pt_words) >= 3

spell = enchant.Dict("pt_BR")

In [ ]:
all_data = load_dataset("clips/mqa", language="pt", cache_dir=cache_dir)
filtered_data = all_data.filter(lambda example: '?' in example['name'] and is_portuguese(example['name']))

Generating train split: 0 examples [00:00, ? examples/s]

Dataset mqa downloaded and prepared to /content/drive/MyDrive/QA_pt/cache_huggingface/clips___mqa/pt-all-question-language=pt/0.0.0/7eda4cdcbd6f009259fc516f204d776915a5f54ea2ad414c3dcddfaacd4dfe0b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset mqa downloaded and prepared to /content/drive/MyDrive/QA_pt/cache_huggingface/clips___mqa/pt-faq-question-language=pt,scope=faq/0.0.0/7eda4cdcbd6f009259fc516f204d776915a5f54ea2ad414c3dcddfaacd4dfe0b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset mqa downloaded and prepared to /content/drive/MyDrive/QA_pt/cache_huggingface/clips___mqa/pt-cqa-question-language=pt,scope=cqa/0.0.0/7eda4cdcbd6f009259fc516f204d776915a5f54ea2ad414c3dcddfaacd4dfe0b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
COLUMNS = [
    'id',
    'domain',
    'bucket',
    'question_title',
    'question_text',
    'answer_title',
    'answer_text',
    'is_accepted',
]

def format(data):
  prepro_data = {column: list() for column in COLUMNS}

  for question_idx in range(len(data['id'])):

    for answer_idx, answer in enumerate(data['answers'][question_idx]):
      prepro_data['id'].append(data['id'][question_idx] + '-' + str(answer_idx))


      for column in ['domain', 'bucket']:
        prepro_data[column].append(data[column][question_idx])

      prepro_data['question_title'].append(data['name'][question_idx])
      prepro_data['question_text'].append(data['text'][question_idx])
      prepro_data['answer_title'].append(answer['name'])
      prepro_data['answer_text'].append(answer['text'])
      prepro_data['is_accepted'].append(answer['is_accepted'])
  
  return prepro_data

prepro_data = filtered_data.map(
    format,
    batched=True,
    remove_columns=[
        'text',
        'name',
        'answers',
    ]
)

  0%|          | 0/5477 [00:00<?, ?ba/s]

In [ ]:
import pycountry

PT_COUNTRIES =  [
    'brazil',
    'portugal',
    'angola',
    'cabo verde',
    'guinea-bissau',
    'mozambique',
    'timor-leste'
]

def is_portuguese(domain):
    end = domain.split('.')[-1]

    if len(end) != 2:
        return True

    language = pycountry.countries.get(alpha_2=end)

    if not language:
        return True

    return language.name.lower() in PT_COUNTRIES

prepro_data = prepro_data.filter(lambda examples: is_portuguese(examples['domain'])) 

In [ ]:
prepro_data.save_to_disk('prepro_data')

In [ ]:
df = prepro_data['train'].to_pandas()
df.to_csv('QA_PT.tsv', sep='\t', index=None)

In [ ]:
%ls

cache_huggingface/  prepro_data/  QA_PT.tsv


In [ ]:
with open('QA_PT.tsv', 'r') as f:
    data = f.read()

data = data.replace('\r', '')

with open('QA_PT.tsv', 'w') as f:
    f.write(data)

In [ ]:
import pandas as pd

df = pd.read_table('QA_PT.tsv')
df.fillna('-', inplace=True)
df.to_csv('train.tsv', index=None, sep='\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
